Scraping date: 2023/09/21

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [4]:
url = 'https://recalls-rappels.canada.ca/en/search/site?search_api_fulltext=&archived=1&f%5B0%5D=category%3A144&page=0%27)'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')

In [5]:
print(soup)

<!DOCTYPE html>

<!--[if lt IE 9]><html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# " class="no-js"><![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<script async="" src="https://www.googletagmanager.com/

In [25]:
page_links = soup.find_all('div', class_ = 'search-result views-row')
page_num
#<div class="search-result views-row"><div class="views-field views-field-field-recall-type"><span class="field-content"><img src="/sites/default/files/2021-11/icon-food.svg" alt="&quot; &quot;" class="home-recent-icon">

In [26]:
'https://recalls-rappels.canada.ca'+ page_links[0].find('a').get('href')

'https://recalls-rappels.canada.ca/en/alert-recall/littleoak-brand-infant-formula-products-not-approved-sale-canada-0'

In [27]:
page_links[0].find('a').text

'LittleOak brand infant formula products not approved for sale in Canada'

### Extracting additional information

In [28]:
page_links[0].find('div', class_='search-excerpt-wrapper').text.strip().replace('\n\n', ' ')

'The Canadian Food Inspection Agency (CFIA) is warning the public not to consume the affected products.\xa0The company has not submitted the required pre-market submission, these\xa0products have not been evaluated to determine whether they meet…'

In [29]:
#to get the date attribute
page_links[0].find('span', class_='ar-type').text.strip().split('|')[-1].strip()

'2023-09-20'

In [52]:
recalled = []

for i in range(1, 326):
    url = f'https://recalls-rappels.canada.ca/en/search/site?search_api_fulltext=&archived=1&f%5B0%5D=category%3A144&page=0%2C{i}'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    page_links = soup.find_all('div', class_='search-result')
    
    for link in page_links:
        notification = {
            'link': 'https://recalls-rappels.canada.ca' + link.find('a').get('href'),
            'title': link.find('a').text,
            'recall_info': link.find('div', class_='search-excerpt-wrapper').text.strip().replace('\n\n', ' '),
            'date': link.find('span', class_='ar-type').text.strip().split('|')[-1].strip()
        }
        recalled.append(notification)

In [53]:
recalls = pd.DataFrame(recalled)
recalls.tail()

,link,title,recall_info,date
4870,https://recalls-rappels.canada.ca/en/alert-rec...,Archive - Health Canada Reminds Canadians ab...,Archive - Health Canada Reminds Canadians abou...,2006-08-01
4871,https://recalls-rappels.canada.ca/en/alert-rec...,Archive - Health Canada raises concerns abou...,Archive - Health Canada raises concerns about ...,2006-07-12
4872,https://recalls-rappels.canada.ca/en/alert-rec...,Archive - Health Canada cautions air travell...,Archive - Health Canada cautions air traveller...,2006-06-30
4873,https://recalls-rappels.canada.ca/en/alert-rec...,Archive - Health Canada reminds Canadians ab...,Archive - Health Canada reminds Canadians abou...,2006-06-21
4874,https://recalls-rappels.canada.ca/en/alert-rec...,Archive – Health Canada Releases Final Resul...,Archive – Health Canada Releases Final Results...,2006-06-09


In [54]:
recalls.iloc[660]

link           https://recalls-rappels.canada.ca/en/alert-rec...
title            Adar brand Schmaltz Herring in Oil recalled ...
recall_info    Adar brand Schmaltz Herring in Oil recalled du...
date                                                  2020-10-14
Name: 660, dtype: object

In [55]:
#checking for missing values if any
recalls.isnull().sum()

link           0
title          0
recall_info    0
date           0
dtype: int64

### Saving scrapped recall info to a csv file

Scrapped items, in total 4891 of 4891 items

In [56]:
recalls.to_csv('recall_links_2023.csv')

### Reference

In [62]:
print(type(page_links))
print(type(page_links[0]))
print(len(page_links))

<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>
15


In [112]:
#load collecetd Data
recalls = pd.read_csv(r'recall_links_2023.csv')
recalls.head()

,Unnamed: 0,link,title,recall_info,date
0,0,https://recalls-rappels.canada.ca/en/alert-recall/frozen-head-shell-vannamei-shrimp-recalled-due-undeclared-sulphites,Frozen Head On Shell On Vannamei Shrimp recalled due to undeclared sulphites,Frozen Head On Shell On Vannamei Shrimp recalled due to undeclared sulphites.,2023-08-16
1,1,https://recalls-rappels.canada.ca/en/alert-recall/mateina-brand-yerba-mate-sparkling-energy-infusions-recalled-due-caffeine-labelling,Mateína brand Yerba Mate Sparkling Energy Infusions recalled due to caffeine labelling requirements,The affected products are being recalled from the marketplace due to non-compliance regarding supplemented food cautionary labelling requirements to avoid excessive consumption of caffeine.,2023-08-11
2,2,https://recalls-rappels.canada.ca/en/alert-recall/soft-serve-go-brand-frozen-dessert-cups-recalled-due-listeria-monocytogenes,Soft Serve on the Go brand frozen dessert cups recalled due to Listeria monocytogenes,The affected products are being recalled from the marketplace due to possible Listeria monocytogenes contamination.,2023-08-10
3,3,https://recalls-rappels.canada.ca/en/alert-recall/fish-maw-recalled-due-undeclared-sulphites,Fish Maw recalled due to undeclared sulphites,Fish Maw recalled due to undeclared sulphites.,2023-08-01
4,4,https://recalls-rappels.canada.ca/en/alert-recall/sss-brand-mushroom-enoki-recalled-due-listeria-monocytogenes,SSS brand Mushroom (enoki) recalled due to Listeria monocytogenes,The affected product is being recalled from the marketplace due to possible Listeria monocytogenes contamination.,2023-07-28


In [113]:
#dropping first column
recalls.drop(recalls.columns[0], axis=1, inplace=True)

#dropping rows that contain advisories instead of recalls
recalls.drop(recalls.index[4905:], axis=0, inplace=True)

#dropping rows that do not contain dates as they are advisories instead of recalls
non_recalls_index = recalls[recalls['date'].str.contains('[a-z]')].index
recalls.drop(non_recalls_index, axis=0, inplace=True)

#transforming date column to datetime format
recalls['date'] = pd.to_datetime(recalls.date, format='%Y-%m-%d')

recalls['title'] = recalls['title'].str.strip()
recalls.head()

,link,title,recall_info,date
0,https://recalls-rappels.canada.ca/en/alert-recall/frozen-head-shell-vannamei-shrimp-recalled-due-undeclared-sulphites,Frozen Head On Shell On Vannamei Shrimp recalled due to undeclared sulphites,Frozen Head On Shell On Vannamei Shrimp recalled due to undeclared sulphites.,2023-08-16
1,https://recalls-rappels.canada.ca/en/alert-recall/mateina-brand-yerba-mate-sparkling-energy-infusions-recalled-due-caffeine-labelling,Mateína brand Yerba Mate Sparkling Energy Infusions recalled due to caffeine labelling requirements,The affected products are being recalled from the marketplace due to non-compliance regarding supplemented food cautionary labelling requirements to avoid excessive consumption of caffeine.,2023-08-11
2,https://recalls-rappels.canada.ca/en/alert-recall/soft-serve-go-brand-frozen-dessert-cups-recalled-due-listeria-monocytogenes,Soft Serve on the Go brand frozen dessert cups recalled due to Listeria monocytogenes,The affected products are being recalled from the marketplace due to possible Listeria monocytogenes contamination.,2023-08-10
3,https://recalls-rappels.canada.ca/en/alert-recall/fish-maw-recalled-due-undeclared-sulphites,Fish Maw recalled due to undeclared sulphites,Fish Maw recalled due to undeclared sulphites.,2023-08-01
4,https://recalls-rappels.canada.ca/en/alert-recall/sss-brand-mushroom-enoki-recalled-due-listeria-monocytogenes,SSS brand Mushroom (enoki) recalled due to Listeria monocytogenes,The affected product is being recalled from the marketplace due to possible Listeria monocytogenes contamination.,2023-07-28


In [114]:
recalls_before_2022 = recalls[recalls['date'] < '2022-01-01']

#to check if we properly retrieved the records
if not recalls_before_2022.empty:
    print(recalls_before_2022.iloc[0])
else:
    print("No records before 2022")

link                                                                                                                                          https://recalls-rappels.canada.ca/en/alert-recall/certain-save-foods-brand-combo-meals-recalled-due-salmonella
title                                                                                                                                                                                     Certain Save on Foods brand combo meals recalled due to Salmonella
recall_info    Save-on-Foods is recalling certain Save on Foods brand combo meals from the marketplace due to possible Salmonella contamination.\r\n\r\nThe recalled products have been sold in British Columbia, Alberta, Saskatchewan, Manitoba and Yukon.
date                                                                                                                                                                                                                                     2021-12-

In [115]:
recalls.columns

Index(['link', 'title', 'recall_info', 'date'], dtype='object')

This suggests that we have successfully retrived the the required information for analysis

### 1. Extracting recalls from 2023 to 2022

In [116]:
recalls_p_2023_2022 = recalls[recalls['date'] > '2022-12-30'].reset_index()

titles = []
issues = []
audiences = []
companies = []
categories = []
audiences_2 = []
recall_classes = []

for link in recalls_p_2023_2022['link']:
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    details = soup.find_all('dd', class_='paddingNone')
    details_alt = soup.find_all('div', class_='field--items')

    titles.append(soup.find('h1', class_='gc-thickline').text.strip().replace('\n','') if soup.find('h1', class_='gc-thickline') else None)
    
    if details:
        issues.append(details[3].text.strip())
        categories.append(details[3].text.strip())
        companies.append(details[6].text.strip())
        audiences_2.append(details[7].text.strip())
        recall_classes.append(details[4].text.strip())
    elif details_alt:
        issues.append(details_alt[1].find('div', class_='field--item').text.strip() if len(details_alt) > 1 and details_alt[1].find('div', class_='field--item') else None)
        audiences.append(details_alt[4].find('div', class_='field--item').text.strip() if len(details_alt) > 4 and details_alt[4].find('div', class_='field--item') else None)
        categories.append(details_alt[1].find('div', class_='field--item').text.strip() if len(details_alt) > 1 and details_alt[1].find('div', class_='field--item') else None)
        companies.append(details_alt[2].find('div', class_='field--item').text.strip() if len(details_alt) > 2 and details_alt[2].find('div', class_='field--item') else None)
        audiences_2.append(details_alt[3].find('div', class_='field--item').text.strip() if len(details_alt) > 3 and details_alt[3].find('div', class_='field--item') else None)
        recall_classes.append(details_alt[5].find('div', class_='field--item').text.strip() if len(details_alt) > 5 and details_alt[5].find('div', class_='field--item') else None)
    else:
        issues.append(None)
        audiences.append(None)
        categories.append(None)
        companies.append(None)
        audiences_2.append(None)
        recall_classes.append(None)

In [119]:
recalls_ca = pd.DataFrame({'title': titles, 'issue':issues, 'audience': audiences, 'company': companies, 'category': categories,
                              'audience_2': audiences_2, 'recall_class': recall_classes})

merged_01 = pd.merge(recalls_p_2023_2022, recalls_ca, left_on='title', right_on='title', how='inner')
merged_01.drop_duplicates('title', inplace=True)

merged_01.to_csv('recalls_2023.csv')

In [106]:
recalls.shape

(4875, 4)

In [103]:
merged_01.columns

Index(['link', 'title', 'recall_info', 'date', 'issue', 'audience', 'company',
       'category', 'audience_2', 'recall_class'],
      dtype='object')

In [104]:
recalls_ca.columns

Index(['title', 'issue', 'audience', 'company', 'category', 'audience_2',
       'recall_class'],
      dtype='object')

### Now extracting recalls information from 2022 till 2011

In [122]:
recalls_2011_01_04 = recalls[recalls['date'] < '2021-10-20'].reset_index()
recalls_2011_01_04.shape

(4454, 5)

In [123]:
#extracting recalls for each and every mentioned year
recalls_2022 = recalls[(recalls['date'] < '2023-01-01') & (recalls['date'] > '2021-01-01')].reset_index()
print('2022 recalls: ' + str(recalls_2022.shape[0]))

recalls_2021 = recalls[(recalls['date'] < '2021-10-20') & (recalls['date'] > '2021-01-01')].reset_index()
print('2021 recalls: ' + str(recalls_2021.shape[0]))

recalls_2020 = recalls[(recalls['date'] < '2021-01-01') & (recalls['date'] >= '2020-01-01')].reset_index()
print('2020 recalls: ' + str(recalls_2020.shape[0]))

recalls_2019 = recalls[(recalls['date'] < '2020-01-01') & (recalls['date'] >= '2019-01-01')].reset_index()
print('2019 recalls: ' + str(recalls_2019.shape[0]))

recalls_2018 = recalls[(recalls['date'] < '2019-01-01') & (recalls['date'] >= '2018-01-01')].reset_index()
print('2018 recalls: ' + str(recalls_2018.shape[0]))

recalls_2017 = recalls[(recalls['date'] < '2018-01-01') & (recalls['date'] >= '2017-01-01')].reset_index()
print('2017 recalls: ' + str(recalls_2017.shape[0]))

recalls_2016 = recalls[(recalls['date'] < '2017-01-01') & (recalls['date'] >= '2016-01-01')].reset_index()
print('2016 recalls: ' + str(recalls_2016.shape[0]))

recalls_2015 = recalls[(recalls['date'] < '2016-01-01') & (recalls['date'] >= '2015-01-01')].reset_index()
print('2015 recalls: ' + str(recalls_2015.shape[0]))

recalls_2014 = recalls[(recalls['date'] < '2015-01-01') & (recalls['date'] >= '2014-01-01')].reset_index()
print('2014 recalls: ' + str(recalls_2014.shape[0]))

recalls_2013 = recalls[(recalls['date'] < '2014-01-01') & (recalls['date'] >= '2013-01-01')].reset_index()
print('2013 recalls: ' + str(recalls_2013.shape[0]))

recalls_2012 = recalls[(recalls['date'] < '2013-01-01') & (recalls['date'] >= '2012-01-01')].reset_index()
print('2012 recalls: ' + str(recalls_2012.shape[0]))

recalls_2011 = recalls[(recalls['date'] < '2012-01-01') & (recalls['date'] >= '2011-01-01')].reset_index()
print('2011 recalls: ' + str(recalls_2011.shape[0]))

2022 recalls: 484
2021 recalls: 187
2020 recalls: 231
2019 recalls: 363
2018 recalls: 305
2017 recalls: 353
2016 recalls: 415
2015 recalls: 401
2014 recalls: 641
2013 recalls: 472
2012 recalls: 524
2011 recalls: 476


In [124]:
def extract_recalls(year, dataframe):
    titles = []
    issues = []
    audiences = []
    companies = []
    distributions = []
    recall_classes = []

    for link in dataframe['link']:
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        details = soup.find_all('dd', class_='paddingNone')
        details_alt = soup.find_all('div', class_='field--items')

        title = soup.find('h1', class_='gc-thickline')
        if title:
            titles.append(title.text.strip().replace('\n',''))
        else:
            titles.append(None)

        if len(details) > 3:
            issues.append(details[3].text.strip())
        elif len(details_alt) > 1 and details_alt[1].find('div', class_='field--item'):
            issues.append(details_alt[1].find('div', class_='field--item').text)
        else:
            issues.append(None)

        if len(details) > 8:
            audiences.append(details[8].text.strip())
        elif len(details_alt) > 4 and details_alt[4].find('div', class_='field--item'):
            audiences.append(details_alt[4].find('div', class_='field--item').text)
        else:
            audiences.append(None)

        if len(details) > 6:
            companies.append(details[6].text.strip())
        else:
            companies.append(None)

        if len(details) > 7:
            distributions.append(details[7].text.strip())
        else:
            distributions.append(None)

        recall_class = details[4] if len(details) > 4 else None
        if recall_class:
            recall_classes.append(recall_class.text.strip())
        elif len(details_alt) > 5 and details_alt[5].find('div', class_='field--item'):
            recall_classes.append(details_alt[5].find('div', class_='field--item').text)
        else:
            recall_classes.append(None)

    recalls_ca = pd.DataFrame({'title': titles, 'issue':issues, 'audience': audiences, 'company': companies,
                                  'distribution': distributions, 'recall_class': recall_classes})

    merged_02 = pd.merge(dataframe, recalls_ca, left_on='title', right_on='title', how='inner')
    merged_02.drop_duplicates('title', inplace=True)
    
    return merged_02.to_csv(f'recalls-{year}.csv')

In [126]:
#calling the function and saving csv files
extract_recalls('2022', recalls_2022)

In [127]:
extract_recalls('2021', recalls_2021)